# Testando o modelo do detector de emoções

**Etapa 1 - Importando as bibliotecas**

In [ ]:
import cv2
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow
import zipfile

In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import tensorflow
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
tensorflow.__version__

'2.12.0'

**Etapa 2 - Conectando com o Drive e acessando os arquivos**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/MyDrive/Material_curso.zip"
zip_object = zipfile.ZipFile(file = path, mode = "r")
zip_object.extractall('./')
zip_object.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/gdrive/MyDrive/Material_curso.zip' mode='r'>>

In [ ]:
imagem = cv2.imread('/content/Material_curso/testes/mulher_feliz.jpg')
cv2_imshow(imagem)

In [ ]:
imagem.shape

# Testando o Detector

**Carregamento dos modelos**

In [ ]:
cascade_faces = "Material_curso/haarcascade_frontalface_default.xml"
caminho_modelo = "Material_curso/modelo_01_expressoes.h5"
face_detection = cv2.CascadeClassifier(cascade_faces)
classificador_emocoes = load_model(caminho_modelo, compile = False)
expressoes = ["Raiva", "Nojo", "Medo", "Feliz", "Triste", "Surpreso", "Neutro"]

**Detecção de faces**

In [ ]:
original = imagem.copy()
faces = face_detection.detectMultiScale(original, scaleFactor = 1.1,
                                        minNeighbors = 3, minSize = (20,20))

In [ ]:
faces

In [ ]:
len(faces)

In [ ]:
faces.shape

**Extração do ROI (região de interesse)**

In [ ]:
cinza = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
cv2_imshow(cinza)

In [ ]:
cinza.shape

In [ ]:
roi = cinza[68:68 + 200, 446:446 + 200]

In [ ]:
cv2_imshow(roi)

In [ ]:
roi.shape

(200, 200)

In [ ]:
roi

In [ ]:
roi = cv2.resize(roi, (48, 48))
cv2_imshow(roi)

In [ ]:
roi.dtype

In [ ]:
roi = roi.astype('float')
roi.dtype

In [ ]:
roi

In [ ]:
roi = roi / 255

In [ ]:
roi

In [ ]:
roi = img_to_array(roi)

In [ ]:
roi

In [ ]:
roi.shape

In [ ]:
roi = np.expand_dims(roi, axis = 0)

In [ ]:
roi.shape

# Previsões

In [ ]:
preds = classificador_emocoes.predict(roi)[0]

In [ ]:
preds

In [ ]:
len(preds)

In [ ]:
emotion_probability = np.max(preds)
emotion_probability

In [ ]:
preds.argmax()

In [ ]:
label = expressoes[preds.argmax()]
label

# Resultados

In [ ]:
cv2.putText(original, label, (446, 68 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65,
            (0, 0, 255), 2, cv2.LINE_AA)
cv2.rectangle(original, (446, 68), (446 + 200, 68 + 200), (0, 0, 255), 2)
cv2_imshow(original)

In [ ]:
probabilidades = np.ones((250,300,3), dtype= 'uint8') * 255 #inteiro
probabilidades

In [ ]:
probabilidades.shape

In [ ]:
cv2_imshow(original)
if len(faces) == 1:
  for (i, (emotion, prob)) in enumerate(zip(expressoes, preds)):
    #print(i, emotion, prob)
    text = "{}: {:.2f}%".format(emotion, prob * 100)
    w = int(prob * 300)
    cv2.rectangle(probabilidades, (7, (i * 35) + 5), (w, (i * 35) + 35), (200, 250, 20), -1)
    cv2.putText(probabilidades, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 0), 1, cv2.LINE_AA)
cv2_imshow(probabilidades)